## Cleaning our data 

### Imporing modules and detasets 

In [1]:
# Importing modules 
import pandas as pd 

In [2]:
# Importing the datasets needed 
path = "C:/Users/fredr/OneDrive/Documents/Master/Semester_1/Visual_analytics/Project/"
df_all_data = pd.read_csv(path + "gvc_trade_WITS-update.csv")
df_sectors = pd.read_csv(path + "sector-tiva.csv", sep=";") # The seperator for this dataset is ";"
df_countries = pd.read_csv(path + "gvc-countries.csv")

### Filtering the dataset 

In [3]:
# Create a list of the columns to be dropped
uneccesary_col = ["gtrade_fin", "gtrade_int", "traditional_trade_int", "traditional_trade_fin", "gvcbp", "gvcfp", "gvcmix"]

# Drop the specified columns
df_data = df_all_data.drop(uneccesary_col, axis=1)

In [4]:
# Filtering dat for a range of years 

# Creating a index of year that we want dropped 
before95_index = df_data[df_data["t"] < 1995].index 

# Dropping the index 
df_data.drop(before95_index, inplace = True)

In [5]:
# Specify the names of the columns to use for the merge
left_on = ["sect", "source"]
right_on = ["sect", "source"]
# Perform the merge
df_filtered = pd.merge(df_data, df_sectors, left_on=left_on, right_on=right_on, how = "left")

# Drop uncessesary columns
df_filtered.drop(["sect"], axis=1, inplace=True)

In [6]:
# Change country abbreviation to ful name (eg. GER --> Germany)

# Use a dictionary comprehension to create a dictionary from the dataframe
dic_country_name = {row["country"]: row["country_name"] for _, row in df_countries.iterrows()}

# change country abbreviation to full name in exp and imp column
for col in ["exp", "imp"]:
    df_filtered[col] = df_filtered[col].map(dic_country_name)

### Cleaning the data 

In [7]:
# Getting a look at the filtered data 
df_filtered.sample(15)

,exp,imp,t,source,gtrade,traditional_trade,gvc,category
10988850,Suriname,Cayman Islands,2011,eora,0.004564,0.003120,0.001444,NaN
24745815,Brazil,Japan,2007,tiva,3.605302,3.073243,0.532059,Human health and social work activities
5010085,Haiti,Morocco,2007,eora,0.013367,0.008678,0.004689,NaN
30893842,Switzerland,Senegal,2006,tiva,0.796845,0.591932,0.204913,Telecommunications
6373021,Seychelles,Taiwan,1996,eora,0.017408,0.007393,0.010015,NaN
12106561,UAE,New Zealand,1999,eora,0.013360,-0.001220,0.014580,NaN
13951501,Andorra,Bolivia,1995,eora,0.004879,0.003022,0.001858,NaN
3454394,Papua New Guinea,Vanuatu,1998,eora,0.004221,0.002283,0.001938,NaN
9646797,Croatia,Belize,2012,eora,0.011999,0.006736,0.005263,NaN
31002402,Lithuania,Chile,2006,tiva,0.035767,0.019205,0.016562,Paper products and printing


In [29]:
# Cheking for null values 
df_filtered.isnull().sum()

exp                    130638
imp                    130638
t                           0
source                      0
gtrade                      0
traditional_trade           0
gvc                         0
category             24307114
dtype: int64

In [30]:
# Cheking if the source is correct 
df_filtered.groupby("source").count()

,exp,imp,t,gtrade,traditional_trade,gvc,category
source,,,,,,,
adb,2083725,2083725,2083725,2083725,2083725,2083725,0
eora,19503666,19503666,19503666,19503666,19503666,19503666,0
tiva,6846840,6846840,6936930,6936930,6936930,6936930,6936930
wiodlr,89700,89700,93288,93288,93288,93288,0
wiodn,1589280,1589280,1626240,1626240,1626240,1626240,0
wiodo,1000195,1000195,1000195,1000195,1000195,1000195,0


When we check for null values, we have 130,638 null values in "exp" and "imp". This is because TIVA mainly focuses on OECD countries and their repsected trading partners, and therefore, the null values is all the other countires in the world. We, therefore, categorise these as "Rest of the World". Furthermore, we have null values in the category column. This is mainly becuase we use "left join" when we merge the sectors with our main dataset. Since we will mainly focus on TIVA we will, therefore, dropp all the other sectors. 

In [31]:
# Chaning the null values to "Rest of the World"
df_filtered["exp"].fillna("Rest of the World", inplace = True)
df_filtered["imp"].fillna("Rest of the World", inplace = True)

In [34]:
# Defining the values we want to be droppped 
Dropp_sectors = ["adb", "eora", "wiodlr", "wiodn", "wiodo"]

# Dropping the rows that contain these values
df_clean = df_filtered[df_filtered.source.isin(Dropp_sectors) == False]

In [37]:
# Chainging "Viet Nam" to Vietnam
df_clean["exp"] = df_clean["exp"].replace(["Viet Nam"], "Vietnam")
df_clean["imp"] = df_clean["imp"].replace(["Viet Nam"], "Vietnam")

C:\Users\fredr\AppData\Local\Temp\ipykernel_20400\424035730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["exp"] = df_clean["exp"].replace(["Viet Nam"], "Vietnam")
C:\Users\fredr\AppData\Local\Temp\ipykernel_20400\424035730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["imp"] = df_clean["imp"].replace(["Viet Nam"], "Vietnam")


In [38]:
# Chainging the column names 
df_clean.rename(columns = {"exp":"Export", "imp":"Import", "t":"Year", "source":"Source", "gtrade":"Gross Trade", "traditional_trade":"Traditional Trade", "gvc":"GVC", "category":"Category"}, inplace = True)

C:\Users\fredr\AppData\Local\Temp\ipykernel_20400\89940070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.rename(columns = {"exp":"Export", "imp":"Import", "t":"Year", "source":"Source", "gtrade":"Gross Trade", "traditional_trade":"Traditional Trade", "gvc":"GVC", "category":"Category"}, inplace = True)


In [39]:
# Removing where export and import is the same country 
df_clean = df_clean[df_clean["Export"] != df_clean["Import"]]

In [40]:
# Looking at the clean dataframe
df_clean.sample(15)

,Export,Import,Year,Source,Gross Trade,Traditional Trade,GVC,Category
30297105,Ireland,New Zealand,2011,tiva,0.039434,0.036008,0.003426,Human health and social work activities
25391542,Austria,USA,2013,tiva,1085.041539,584.443748,500.597791,"Machinery and equipment, nec"
26810492,Hong Kong,Israel,2002,tiva,23.183938,12.592138,10.591800,Air transport
27529577,Bangladesh,China,1996,tiva,0.005726,0.004555,0.001171,Other non-metallic mineral products
28969420,Israel,Cambodia,2019,tiva,0.219980,0.195783,0.024197,Real estate activities
25635737,Vietnam,India,2013,tiva,0.630105,0.456063,0.174042,Other service activities
31065955,Finland,Latvia,2020,tiva,6.444260,4.309351,2.134909,Manufacturing nec; repair and installation of ...
29247719,Japan,Slovakia,2000,tiva,4.361217,2.406615,1.954602,Chemical and chemical products
26185550,Australia,Brazil,2014,tiva,0.564677,0.438793,0.125884,Fishing and aquaculture
28147247,Croatia,Germany,2017,tiva,16.522643,8.941191,7.581452,Other non-metallic mineral products


### Saving the cleaned dataset

In [41]:
# Save the clean dataset as a csv file
# df_clean.to_csv("C:/Users/fredr/OneDrive/Documents/Master/Semester_1/Visual_analytics/Project/cleaned_data.csv")